# Reading data from the Load Cell and Conversion in Pressure

<i>© Pierre-Marie Lefeuvre, University of Oslo//Norwegian Water Resources and Energy Directorate</i>

This Notebook gives an introduction to the R routines processing **load cell data**
The "Processing" Directory contains all codes that are used to convert load cell frequency into pressure, then save/load the data in different format and finally to process the pressure data for different analysis.

# 0) Path and libraries

To run all codes, the variable *HOME* and *PATH* in *setwd()* must be changed in each program to your own path as shown below:

In [ ]:
# Set Working Directory (!!! EDIT TO YOUR PATH !!!)
setwd("/Users/PiM/Desktop/NVE_work/Processing/")

You can run this bash code in a terminal to rapidly change the PATH (tested on Mac)

In [ ]:
for file in Processing/*.R
    do 
    sed -i '' -e 's/.*setwd.*/setwd(PATH)/g' $file
    done
    # Common error: in the path, any "/" must be backslashed ("\/") in order to work in the command "sed"
    # ex.: PATH="\/Users\/PiM\/Desktop\/NVE_work\/Processing"

Install the following libraries as you need them to run all the R routines:

In [ ]:
install.packages(c("zoo","chron","hydroTSM","Hmisc","lattice","latticeExtra","signal","MASS","hydroGOF",
                   "RColorBrewer","lubridate","psych","RCurl","quantmod","classInt","aqfig","plyr","scales","ggplot2"))

# 1) Process Raw data

The first program to run is "LoadAllData_subsample_save.R"/"LoadAllData_subsample_save_freq.R".
The year(s) to be processed has to be declared in the program line 47 or 57, respectively such as:

In [ ]:
# Load the Data and combine them
Years       <- seq(2011,2014) # for the period 2011-2014, including 2014
# OR
# Load the Data and combine them
Years       <- 2013           # for one year, i.e. 2013

The two programs will then read, combine, and convert the frequency data for the defined period. Run them with:

In [ ]:
source('LoadAllData_subsample_save.R')      # To convert into pressure
source('LoadAllData_subsample_save_freq.R') # To keep frequency data

The output is a regular time series of pressure/frequency and is saved into files of 2 months in the directory "Data/Zoo/". 
This splitting facilitates the loading and analysis of the load cell data.

# 2) Load the data

Now, that the data are saved and processed, they can be loaded into R using:
- "f_Load_ZooSub_month.R" to load the pressure (MPa) or the frequency (Hz) of the load cell

In [ ]:
#First, you have to load it into R:
source('f_Load_ZooSub_month.R')
#Then you can call the function, the period of interest and the data type as follows:
LC.reg.sub <- Load_ZooSub_month("2000-01-01","2001-01-01","15min_mean",freq=FALSE)

# Show the first six lines of the data and the load cell/column names
head(LC.reg.sub)

This loads the **15 min** interval **pressure** data for the period **2000-2001** and saves it into the variable *LC.reg.sub*. **It is important to use the variable *LC.reg.sub* as it is needed to detect whether the data have already been loaded**. 
If you want to load the raw frequency data instead, replace the flag *freq* to *TRUE* (i.e. *freq=TRUE*)

# 3) Plot

Please first create the directory "Plots" inside "Processing". If you are already in the PATH, run in R:

In [ ]:
dir.create("Plots",showWarnings = FALSE)

3a) To plot, You can use the classic plot function in R. As LC.reg.sub is a zoo object, *plot()* will automatically have the time on the x-axis and Pressure on the y-axis. Each time series will have its own panel.

In [ ]:
plot(LC.reg.sub) # or # plot(LC.reg.sub[,1]) plot only data from the first load cell

# If you want all load cells on one single plot, just add:
plot(LC.reg.sub,plot.type="single")

# To plot points instead of lines (the default for zoo data), add: pch=20 or pch="."
plot(LC.reg.sub,plot.type="single",type="p",pch=".")
# pch gives the type of points to use. Find more types in the R help: ?pch

# The color of the time series is defined by the option "col"
plot(LC.reg.sub,plot.type="single",type="p",pch=".",col=c("red","orange","yellow","cyan","blue"))

3b) The pressure can also be plotted in a nicer format using the *Plot_LCs_Pressure()* function contained in "f_Plot_LCs.R". To access it, run: 

In [ ]:
source('f_Plot_LCs.R')

Now, the function is accessible from the Console. The function load internally the LC data, so you don't need "f_Load_ZooSub_month.R". You can run a default example with:

In [ ]:
Plot_LCs_Pressure() 
# that is the same than
Plot_LCs_Pressure(sub.start="2003-07-01",sub.end="2003-07-15",LCname=c("LC97_1","LC97_2"),type="15min_mean",f.plot=FALSE)

Notes: The load cells (LC) are plotted in pairs on subpanels. The number of panels depends on the number of LCs and allows a maximum of three pairs (or three panels).

Input: The start and end of the period is given with the 2 first inputs, then the load cell names, the type of data and whether you want to save the figure:
- *sub.start* and *sub.end* accept date and time in different formats, but the advised format is: *2003-07-01 01:00*.
- The order in *LCname* defines in which order the pairs of LCs are plotted. 
- The by-default data interval is 15 min. 1-min and 1-day data can also be used with *min*, *day_med*.
- The plot is viewed in R with *f.plot* set *FALSE* and saved in a pdf if set to *TRUE*, but then the figure does not appear in R.

3d) Another routine gives you an overview of the entire data, but you need to have data for the period 1992-2014.
The plot is directly saved in a pdf (Fig.3 in Lefeuvre et al., 2015). 

In [ ]:
source('Plot_LC_all.R')

!!! WARNING: it may take a while. The plotting is much faster if the daily values are loaded instead of the 15 min data (change line 59-60).